# Isolated Pad Foundation Design - ACI 318M-25 Example

This example demonstrates the design of an isolated pad foundation using **ACI 318M-25 Building Code Requirements for Structural Concrete (Metric)**.

## Design Standards
- **ACI 318M-25 Chapter 13.1**: Foundations
- **ACI 318M-25 Section 5.3**: Load combinations and strength reduction factors
- **ACI 318M-25 Section 7**: Flexural design
- **ACI 318M-25 Section 22**: Shear and torsion
- **ACI 318M-25 Section 22.6**: Two-way shear (punching shear)

## Problem Statement

Design a square pad foundation for an interior column with the following requirements:

**Column Loads:**
- Dead Load (D): 800 kN
- Live Load (L): 300 kN
- Column Size: 400mm × 400mm

**Foundation Parameters:**
- Allowable soil bearing capacity: 200 kN/m²
- Foundation thickness: 400mm
- Concrete cover: 75mm (per ACI 318M-25 Section 20.5.1.3)

**Material Properties:**
- Concrete: f'c = 30 MPa
- Steel: fy = 420 MPa

In [ ]:
# Install the ACI 318M-25 foundation design package
!pip install FoundationDesign-ACI318

In [ ]:
# Import the ACI 318M-25 foundation design classes
from FoundationDesign import PadFoundationACI318, padFoundationDesignACI318
from FoundationDesign import get_design_info, validate_material_properties
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Design Code Information

First, let's verify we're using the correct design standard and load factors.

In [ ]:
# Display design code information
design_info = get_design_info()
print(f"Design Code: {design_info['design_code']}")
print(f"Standard: {design_info['design_standard']}")
print(f"Version: {design_info['version']}")
print("\nApplicable Chapters:")
for chapter in design_info['applicable_chapters']:
    print(f"  - {chapter}")

print("\nACI 318M-25 Load Factors (Section 5.3.1):")
for load_type, factor in design_info['default_load_factors'].items():
    print(f"  {load_type.replace('_', ' ').title()}: {factor}")

print("\nACI 318M-25 Strength Reduction Factors (Section 5.4.2):")
for element, phi in design_info['default_phi_factors'].items():
    print(f"  φ {element.replace('_', ' ')}: {phi}")

## Step 2: Validate Material Properties

Ensure material properties comply with ACI 318M-25 requirements.

In [ ]:
# Material properties
fc_prime = 30  # MPa - specified compressive strength of concrete
fy = 420       # MPa - specified yield strength of reinforcement

# Validate material properties
validation = validate_material_properties(fc_prime, fy)
print(f"Material validation: {'PASS' if validation['valid'] else 'FAIL'}")

if validation['errors']:
    print("\nErrors:")
    for error in validation['errors']:
        print(f"  - {error}")

if validation['warnings']:
    print("\nWarnings:")
    for warning in validation['warnings']:
        print(f"  - {warning}")

print(f"\nUsing: f'c = {fc_prime} MPa, fy = {fy} MPa")

## Step 3: Foundation Sizing

Size the foundation based on service loads and allowable bearing pressure.

In [ ]:
# Column and load data
dead_load = 800  # kN
live_load = 300  # kN
total_service_load = dead_load + live_load  # kN

# Column dimensions
column_length = 400  # mm
column_width = 400   # mm

# Foundation parameters
allowable_bearing = 200  # kN/m²
foundation_thickness = 400  # mm
concrete_unit_weight = 24   # kN/m³
soil_unit_weight = 18       # kN/m³
soil_depth = 700           # mm

# Estimate foundation self-weight and surcharge
# Assume square foundation initially
foundation_size_estimate = 2500  # mm (initial guess)

foundation_self_weight = (foundation_size_estimate**2 * foundation_thickness / 1e9) * concrete_unit_weight
surcharge_load = (foundation_size_estimate**2 * soil_depth / 1e9) * soil_unit_weight
total_load_estimate = total_service_load + foundation_self_weight + surcharge_load

# Required foundation area
required_area = total_load_estimate / allowable_bearing  # m²
foundation_size = int(np.sqrt(required_area * 1e6))  # mm

# Round up to nearest 50mm
foundation_size = int(np.ceil(foundation_size / 50) * 50)

print(f"Service loads: D = {dead_load} kN, L = {live_load} kN")
print(f"Total service load = {total_service_load} kN")
print(f"Estimated foundation self-weight = {foundation_self_weight:.1f} kN")
print(f"Estimated surcharge load = {surcharge_load:.1f} kN")
print(f"Total estimated load = {total_load_estimate:.1f} kN")
print(f"Required foundation area = {required_area:.2f} m²")
print(f"Foundation size: {foundation_size} mm × {foundation_size} mm")

## Step 4: Create Foundation Object

Create the foundation analysis object using ACI 318M-25 parameters.

In [ ]:
# Create foundation object with ACI 318M-25 load factors
foundation = PadFoundationACI318(
    foundation_length=foundation_size,        # mm
    foundation_width=foundation_size,         # mm
    column_length=column_length,              # mm
    column_width=column_width,                # mm
    col_pos_xdir=foundation_size/2,          # mm (centered)
    col_pos_ydir=foundation_size/2,          # mm (centered)
    soil_bearing_capacity=allowable_bearing,  # kN/m²
    uls_strength_factor_dead=1.2,            # ACI 318M-25 Section 5.3.1
    uls_strength_factor_live=1.6,            # ACI 318M-25 Section 5.3.1
    uls_strength_factor_wind=1.0,            # ACI 318M-25 Section 5.3.1
    phi_flexure=0.9,                         # ACI 318M-25 Section 5.4.2.1
    phi_shear=0.75,                          # ACI 318M-25 Section 5.4.2.3
)

print(f"Foundation created: {foundation_size}mm × {foundation_size}mm × {foundation_thickness}mm")
print(f"Foundation area: {foundation.area_of_foundation()/1e6:.2f} m²")
print(f"Column position: ({foundation.col_pos_xdir}, {foundation.col_pos_ydir}) mm")
print(f"\nACI 318M-25 Load Factors Applied:")
print(f"  Dead load factor: {foundation.uls_strength_factor_dead}")
print(f"  Live load factor: {foundation.uls_strength_factor_live}")
print(f"  φ flexure: {foundation.phi_flexure}")
print(f"  φ shear: {foundation.phi_shear}")

## Step 5: Apply Loads

Apply column loads and foundation self-weight.

In [ ]:
# Apply column loads
foundation.column_axial_loads(
    dead_axial_load=dead_load,  # kN
    live_axial_load=live_load,  # kN
    wind_axial_load=0           # kN
)

# Calculate foundation self-weight and surcharge
foundation.foundation_loads(
    foundation_thickness=foundation_thickness,
    soil_depth_abv_foundation=soil_depth,
    soil_unit_weight=soil_unit_weight,
    concrete_unit_weight=concrete_unit_weight
)

# Display load summary
service_load = foundation.total_force_Z_dir_service()
ultimate_load = foundation.total_force_Z_dir_ultimate()

print("Load Summary:")
print(f"  Column dead load: {dead_load} kN")
print(f"  Column live load: {live_load} kN")
print(f"  Foundation self-weight: {foundation._foundation_self_weight:.1f} kN")
print(f"  Surcharge load: {foundation._surcharge_load:.1f} kN")
print(f"\nTotal service load: {service_load:.1f} kN")
print(f"Total ultimate load (ACI 318M-25): {ultimate_load:.1f} kN")
print(f"Load factor applied: {ultimate_load/service_load:.2f}")

## Step 6: Bearing Pressure Check

Check foundation bearing pressure against allowable soil capacity.

In [ ]:
# Check bearing pressure at service loads
bearing_check = foundation.bearing_pressure_check_service()

print("Bearing Pressure Check (Service Loads):")
print(f"  Applied pressure: {bearing_check['bearing_pressure']} kN/m²")
print(f"  Allowable pressure: {bearing_check['allowable_pressure']} kN/m²")
print(f"  Utilization ratio: {bearing_check['utilization_ratio']:.3f}")
print(f"  Status: {bearing_check['check_status']}")

if bearing_check['check_status'] == 'PASS':
    print(f"\n✓ Foundation size is adequate for bearing pressure")
else:
    print(f"\n✗ Foundation size needs to be increased")

## Step 7: Complete Foundation Design

Perform complete foundation design including flexural and shear design per ACI 318M-25.

In [ ]:
# Perform complete foundation design per ACI 318M-25
design_results = padFoundationDesignACI318(
    fdn_analysis=foundation,
    concrete_grade=fc_prime,           # f'c = 30 MPa
    steel_grade=fy,                    # fy = 420 MPa
    foundation_thickness=foundation_thickness,
    soil_depth_abv_foundation=soil_depth,
    steel_cover=75,                    # mm - per ACI 318M-25 Section 20.5.1.3
    bar_dia_x=16,                      # mm
    bar_dia_y=16,                      # mm
)

print("=" * 60)
print("FOUNDATION DESIGN RESULTS - ACI 318M-25")
print("=" * 60)

print(f"\nDesign Code: {design_results['design_code']}")

# Foundation geometry
geometry = design_results['foundation_geometry']
print(f"\nFoundation Geometry:")
print(f"  Length: {geometry['length']} mm")
print(f"  Width: {geometry['width']} mm")
print(f"  Thickness: {geometry['thickness']} mm")
print(f"  Area: {geometry['area']/1e6:.2f} m²")

# Material properties
materials = design_results['material_properties']
print(f"\nMaterial Properties:")
print(f"  f'c: {materials['fc_prime']} MPa")
print(f"  fy: {materials['fy']} MPa")
print(f"  Cover: {materials['cover']} mm")

# Loads
loads = design_results['loads']
print(f"\nLoad Summary:")
print(f"  Service load: {loads['service_load']:.1f} kN")
print(f"  Ultimate load: {loads['ultimate_load']:.1f} kN")

# Bearing pressure
bearing = design_results['bearing_pressure']
print(f"\nBearing Pressure Check:")
print(f"  Applied: {bearing['bearing_pressure']} kN/m²")
print(f"  Allowable: {bearing['allowable_pressure']} kN/m²")
print(f"  Status: {bearing['check_status']}")

## Step 8: Flexural Design Results

Display flexural reinforcement requirements per ACI 318M-25 Section 7.

In [ ]:
# Flexural design results
flexural = design_results['flexural_design']

print("\nFlexural Design (ACI 318M-25 Section 7):")
print(f"\nX-Direction:")
print(f"  Required As: {flexural['x_direction']['required_As']:.0f} mm²/m")
print(f"  Minimum As: {flexural['x_direction']['minimum_As']:.0f} mm²/m")
print(f"  Status: {flexural['x_direction']['status']}")

print(f"\nY-Direction:")
print(f"  Required As: {flexural['y_direction']['required_As']:.0f} mm²/m")
print(f"  Minimum As: {flexural['y_direction']['minimum_As']:.0f} mm²/m")
print(f"  Status: {flexural['y_direction']['status']}")

# Reinforcement provision (simplified)
print(f"\nReinforcement Provision:")
As_x = max(flexural['x_direction']['required_As'], flexural['x_direction']['minimum_As'])
As_y = max(flexural['y_direction']['required_As'], flexural['y_direction']['minimum_As'])

# Calculate bar spacing for 16mm bars
bar_area_16 = np.pi * (16/2)**2  # mm²
spacing_x = 1000 * bar_area_16 / As_x  # mm
spacing_y = 1000 * bar_area_16 / As_y  # mm

# Round to practical spacings
practical_spacings = [100, 125, 150, 175, 200, 225, 250]
spacing_x_practical = min([s for s in practical_spacings if s <= spacing_x], default=100)
spacing_y_practical = min([s for s in practical_spacings if s <= spacing_y], default=100)

print(f"  X-direction: 16mm bars @ {spacing_x_practical}mm c/c")
print(f"  Y-direction: 16mm bars @ {spacing_y_practical}mm c/c")
print(f"  As provided X: {1000 * bar_area_16 / spacing_x_practical:.0f} mm²/m")
print(f"  As provided Y: {1000 * bar_area_16 / spacing_y_practical:.0f} mm²/m")

## Step 9: Shear Design Results

Display shear design checks per ACI 318M-25 Sections 22.5 and 22.6.

In [ ]:
# Shear design results
shear = design_results['shear_design']

print("\nShear Design (ACI 318M-25 Section 22):")

# Punching shear (Section 22.6)
punching = shear['punching_shear']
print(f"\nPunching Shear (Section 22.6):")
print(f"  Critical perimeter: {punching['critical_section']['perimeter']:.0f} mm")
print(f"  Effective depth: {punching['critical_section']['distance_from_face']:.0f} mm from column face")
print(f"  Applied force: {punching['punching_force']/1000:.1f} kN")
print(f"  Design strength: {punching['design_strength']/1000:.1f} kN")
print(f"  Demand/Capacity: {punching['demand_capacity_ratio']:.3f}")
print(f"  Status: {punching['check_status']}")
print(f"  Governing case: {punching['governing_case']}")

# One-way shear X-direction (Section 22.5)
shear_x = shear['one_way_x']
print(f"\nOne-way Shear X-direction (Section 22.5):")
print(f"  Critical location: {shear_x['critical_location']:.0f} mm from origin")
print(f"  Applied shear: {shear_x['shear_force']/1000:.1f} kN")
print(f"  Design strength: {shear_x['design_strength']/1000:.1f} kN")
print(f"  Demand/Capacity: {shear_x['demand_capacity_ratio']:.3f}")
print(f"  Status: {shear_x['check_status']}")

# One-way shear Y-direction (Section 22.5)
shear_y = shear['one_way_y']
print(f"\nOne-way Shear Y-direction (Section 22.5):")
print(f"  Critical location: {shear_y['critical_location']:.0f} mm from origin")
print(f"  Applied shear: {shear_y['shear_force']/1000:.1f} kN")
print(f"  Design strength: {shear_y['design_strength']/1000:.1f} kN")
print(f"  Demand/Capacity: {shear_y['demand_capacity_ratio']:.3f}")
print(f"  Status: {shear_y['check_status']}")

## Step 10: Design Summary

Final design summary and adequacy check.

In [ ]:
# Design summary
summary = design_results['design_summary']

print("\n" + "="*60)
print("DESIGN SUMMARY - ACI 318M-25")
print("="*60)

print(f"\nFoundation Adequacy: {'PASS' if summary['foundation_adequate'] else 'FAIL'}")

if summary['foundation_adequate']:
    print("\n✓ All design checks satisfy ACI 318M-25 requirements")
    print("\nFinal Design:")
    print(f"  Foundation: {foundation_size}mm × {foundation_size}mm × {foundation_thickness}mm")
    print(f"  Materials: f'c = {fc_prime} MPa, fy = {fy} MPa")
    print(f"  Reinforcement:")
    print(f"    Bottom X: 16mm @ {spacing_x_practical}mm c/c")
    print(f"    Bottom Y: 16mm @ {spacing_y_practical}mm c/c")
    print(f"  Cover: {materials['cover']}mm (ACI 318M-25 Section 20.5.1.3)")
    
    print(f"\nDesign Checks:")
    print(f"  ✓ Bearing pressure: {bearing['utilization_ratio']:.3f} ≤ 1.0")
    print(f"  ✓ Punching shear: {punching['demand_capacity_ratio']:.3f} ≤ 1.0")
    print(f"  ✓ One-way shear X: {shear_x['demand_capacity_ratio']:.3f} ≤ 1.0")
    print(f"  ✓ One-way shear Y: {shear_y['demand_capacity_ratio']:.3f} ≤ 1.0")
    
else:
    print("\n✗ Foundation design does not satisfy all requirements")
    print("\nFailed Checks:")
    if bearing['check_status'] != 'PASS':
        print(f"  ✗ Bearing pressure: {bearing['utilization_ratio']:.3f} > 1.0")
    if punching['check_status'] != 'PASS':
        print(f"  ✗ Punching shear: {punching['demand_capacity_ratio']:.3f} > 1.0")
    if shear_x['check_status'] != 'PASS':
        print(f"  ✗ One-way shear X: {shear_x['demand_capacity_ratio']:.3f} > 1.0")
    if shear_y['check_status'] != 'PASS':
        print(f"  ✗ One-way shear Y: {shear_y['demand_capacity_ratio']:.3f} > 1.0")

print(f"\nDesign completed per ACI 318M-25 Building Code Requirements for Structural Concrete")

## Conclusion

This example demonstrates the complete design process for an isolated pad foundation using **ACI 318M-25** standards. The design includes:

1. **Load Combinations**: Applied per ACI 318M-25 Section 5.3.1
2. **Strength Reduction Factors**: Applied per ACI 318M-25 Section 5.4.2
3. **Flexural Design**: Per ACI 318M-25 Section 7 using Whitney stress block
4. **Minimum Reinforcement**: Per ACI 318M-25 Section 7.6
5. **One-way Shear**: Per ACI 318M-25 Section 22.5
6. **Punching Shear**: Per ACI 318M-25 Section 22.6
7. **Concrete Cover**: Per ACI 318M-25 Section 20.5.1.3

The resulting foundation design satisfies all ACI 318M-25 requirements for structural adequacy and constructability.